In [5]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd

In [2]:
url = 'https://moringaschool.com/courses/'

page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')  # 'html.parser' should be used as the parser


In [3]:
# thelist = ['https://moringaschool.com/courses/software-engineering-mobile/', 'https://moringaschool.com/courses/data-science-full-time-hybrid/', 'https://moringaschool.com/courses/cybersecurity-comptia-security-course/', 'https://moringaschool.com/courses/product-design-ux-ui-part-time-course/', 'https://moringaschool.com/courses/cybersecurity-prep/', 'https://moringaschool.com/courses/data-visualization-with-python/', 'https://moringaschool.com/courses/devops-engineering/', 'https://moringaschool.com/courses/product-design-ux-ui-full-time/', 'https://moringaschool.com/courses/part-time-software-engineering/', 'https://moringaschool.com/courses/data-science-course-part-time/', 'https://moringaschool.com/courses/data-science-course/', 'https://moringaschool.com/courses/software-engineering-course-online/', 'https://moringaschool.com/courses/software-engineering-course/']

In [4]:
# Find all <section> tags on the page
section_tags = soup.find_all('section')

In [5]:
# some more global variables
# a list of courses here
list_of_courses = []

In [6]:
# crawl into course link and get more information

In [7]:
def fetch_additional_course_details(course_link):
    try:
        # Send an HTTP GET request to the course link
        response = requests.get(course_link)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content of the course page
            soup = BeautifulSoup(response.content, 'html.parser')

            # Extract additional information from the course page as needed
            additional_details = {}

            # Find the main tag with class 'text-primary'
            main_elem = soup.find('main', class_='text-primary')

            if main_elem:
                # Find all sections inside the main tag
                sections = main_elem.find_all('section')

                seclen = len(sections)

                print("The course has sections : " + str(seclen) + " : " + course_link)

                if seclen < 13:
                    return None, False

                # Check if there are at least four sections
                # Get the fourth section and retrieve its chunk
                fourth_section = sections[3]
                # additional_details['FAQs'] = fourth_section.text.strip()
                additional_details['FAQs'] = fourth_section.text.strip()
                
                # Check if there are at least six sections
                # Get the sixth section and retrieve its chunk
                sixth_section = sections[5]
                # additional_details['course_details'] = sixth_section.text.strip()
                additional_details['course_details'] = sixth_section.text.strip()

                 # Check if there are at least six sections
                # Get the sixth section and retrieve its chunk
                seventh_section = sections[6]
                # additional_details['prerequisites'] = seventh_section.text.strip()
                additional_details['prerequisites'] = seventh_section.text.strip()

                # Check if there are at least six sections
                # Get the sixth section and retrieve its chunk
                eith_section = sections[8]
                # additional_details['modules'] = eith_section.text.strip()
                additional_details['modules'] = eith_section.text.strip()

                # Check if there are at least six sections
                # Get the sixth section and retrieve its chunk
                ninth_section = sections[8]
                # additional_details['opportunities'] = ninth_section.text.strip()
                additional_details['opportunities'] = ninth_section.text.strip()

            # Add more fields as needed

            return additional_details, True
        else:
            print(f"Failed to fetch additional details for course: {course_link}")
            return None, False
    except Exception as e:
        print(f"An error occurred while fetching additional course details: {str(e)}")
        return None, False

In [8]:
def extract_courses_info(html_chunk):
    # Parse the HTML chunk
    soup = BeautifulSoup(html_chunk, 'html.parser')

    # Find all <div> elements with class 'block' inside the HTML chunk
    course_divs = soup.find_all('div', class_='block')

    course_info_list = []  # Initialize an empty list to store course information

    for course_div in course_divs:
        # Extract course title and description
        title_elem = course_div.find('h1', class_='text-secondary')
        description_elem = course_div.find('div', class_='leading-relaxed')
        link_elem = course_div.find('a', href=True)

        if title_elem and description_elem:
            title = title_elem.text.strip()
            description = description_elem.text.strip()
            link = link_elem['href']

            # call the function and pass the link here
            some_more_data, successfactor = fetch_additional_course_details(link)

            if not successfactor:
                print(f"that course {title} has too little information.")
                continue

            # Create a dictionary to store the course details
            course_info = {
                'Title': title,
                'Description': description,
                'Link': link,
                'Details': some_more_data
            }

            course_info_list.append(course_info)

    return course_info_list

In [9]:
# Check if there's a second <section> tag
if len(section_tags) >= 2:
    second_section = section_tags[1]  # Index 1 corresponds to the second section
    
    # Find the first <div> with class 'ms-container' inside the second section
    ms_container_div = second_section.find('div', class_='ms-container')
    
    if ms_container_div:
        # Find the first <div> inside ms_container_div
        first_div_inside_ms_container = ms_container_div.find('div')
        
        if first_div_inside_ms_container:
            html_chunk = str(first_div_inside_ms_container)
            # print(html_chunk)

            list_of_courses = extract_courses_info(html_chunk)

            print("HOW MANY COURSES ? " + str(len(list_of_courses)))

            # next: break the chunk into a list of courses.
            # each chunk of course would be extracted differently, 
            # and the inforamtion appended onto a list or sth
        else:
            print("No <div> found inside 'ms-container' div.")
    else:
        print("No <div> with class 'ms-container' found inside the second <section>.")
else:
    print("There are not enough <section> tags on the page.")

The course has sections : 12 : https://moringaschool.com/courses/high-school-holiday-tech-bootcamp/
that course High School Holiday Tech Bootcamp has too little information.
The course has sections : 13 : https://moringaschool.com/courses/software-engineering-mobile/
The course has sections : 13 : https://moringaschool.com/courses/data-science-full-time-hybrid/
The course has sections : 13 : https://moringaschool.com/courses/cybersecurity-comptia-security-course/
The course has sections : 13 : https://moringaschool.com/courses/product-design-ux-ui-part-time-course/
The course has sections : 12 : https://moringaschool.com/courses/cybersecurity-prep/
that course Cybersecurity Prep has too little information.
The course has sections : 10 : https://moringaschool.com/courses/data-visualization-with-python/
that course Data Visualization with Python has too little information.
The course has sections : 13 : https://moringaschool.com/courses/devops-engineering/
The course has sections : 13 : 

In [10]:
for i, course in enumerate(list_of_courses):
    print(i + 1, course)

1 {'Title': 'Software Engineering Mobile', 'Description': 'Whether you’re a student, aspiring developer, or industry professional, our comprehensive and up-to-date curriculum is your gateway to creating innovative and impactful mobile experiences.', 'Link': 'https://moringaschool.com/courses/software-engineering-mobile/', 'Details': {'FAQs': 'Introduction\n\n\nIn a world where mobile devices have become an integral part of our daily lives and mobile technology shapes our world, this course is designed to equip individuals with the skills and knowledge necessary to excel in the dynamic field of mobile application development. Whether you’re a student, aspiring developer, or industry professional, our comprehensive and up-to-date curriculum is your gateway to creating innovative and impactful mobile experiences and preparing for a successful career in mobile software engineering.\n\n\n\n\n\n\nWhat is Mobile Development?\n\n\n\nMobile development, often referred to as app development, is 

In [11]:
# A focus on Data Science
course_numbers_to_print = [2, 8]

# Loop through the list of courses and print the selected courses
for i, course in enumerate(list_of_courses):
    if i + 1 in course_numbers_to_print:
        print(i + 1, course)

2 {'Title': 'Data Science Full-time Hybrid', 'Description': 'Are you passionate about data? Do you want to learn how to analyze data, and provide data-driven insights for decisions? This is the course for you. Kickstart the journey to your future career now!', 'Link': 'https://moringaschool.com/courses/data-science-full-time-hybrid/', 'Details': {'FAQs': 'Learn Python for Data Science, Data Analysis, Data Visualization, Data Modelling & Machine Learning in 25 weeks\n\n\nThe credits are rolling on a movie you’ve just concluded watching and Netflix automatically queues the preview for another movie that you might like; You’re doing some online shopping on Jumia and while scrolling down the product description, you notice a section that says “Most popular” that has the same product from different vendors at different prices; this is Data Science at work and businesses are always looking for talented individuals to help them analyze and draw actionable insights from their data in order to 

In [12]:
import csv

# Define the course numbers you want to export
course_numbers_to_export = [2, 8]

# Filter the list to include only the selected courses
selected_courses = [course for i, course in enumerate(list_of_courses) if i + 1 in course_numbers_to_export]

# Specify the CSV file name
csv_file_name = 'data_science.csv'

# Define the fieldnames for the CSV columns
fieldnames = ['Title', 'Description', 'Link', 'FAQs', 'course_details', 'prerequisites', 'modules', 'opportunities']

# Open the CSV file in write mode and write the data
with open(csv_file_name, 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    # Write the header row
    writer.writeheader()

    # Write the selected courses
    for course in selected_courses:
        # Create a new dictionary by merging the 'Details' dictionary with the course dictionary
        course_data = {**course, **course['Details']}
        # Remove the 'Details' key as it is no longer needed
        del course_data['Details']
        writer.writerow(course_data)

print(f'Selected courses have been saved to {csv_file_name}')


Selected courses have been saved to data_science.csv


In [6]:
import requests
from bs4 import BeautifulSoup

def fetch_additional_course_details(course_link):
    try:
        # Send an HTTP GET request to the course link
        response = requests.get(course_link)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content of the course page
            soup = BeautifulSoup(response.content, 'html.parser')

            # Extract additional information from the course page as needed
            additional_details = {}

            # Find the main tag with class 'text-primary'
            main_elem = soup.find('main', class_='text-primary')

            if main_elem:
                # Find all sections inside the main tag
                sections = main_elem.find_all('section')

                seclen = len(sections)

                print("The course has sections : " + str(seclen) + " : " + course_link)

                if seclen < 13:
                    return None, False

                # Check if there are at least four sections
                # Get the fourth section and retrieve its chunk
                fourth_section = sections[3]
                # additional_details['FAQs'] = fourth_section.text.strip()
                additional_details['FAQs'] = fourth_section.text.strip()
                
                # Check if there are at least six sections
                # Get the sixth section and retrieve its chunk
                sixth_section = sections[5]
                # additional_details['course_details'] = sixth_section.text.strip()
                additional_details['course_details'] = sixth_section.text.strip()

                 # Check if there are at least six sections
                # Get the sixth section and retrieve its chunk
                seventh_section = sections[6]
                # additional_details['prerequisites'] = seventh_section.text.strip()
                additional_details['prerequisites'] = seventh_section.text.strip()

                # Check if there are at least six sections
                # Get the sixth section and retrieve its chunk
                eith_section = sections[8]
                # additional_details['modules'] = eith_section.text.strip()
                # additional_details['modules'] = eith_section.text.strip()

                # Check if there are at least six sections
                # Get the sixth section and retrieve its chunk
                ninth_section = sections[8]
                # additional_details['opportunities'] = ninth_section.text.strip()
                # additional_details['opportunities'] = ninth_section.text.strip()

            # Add more fields as needed

            return additional_details, True
        else:
            print(f"Failed to fetch additional details for course: {course_link}")
            return None, False
    except Exception as e:
        print(f"An error occurred while fetching additional course details: {str(e)}")
        return None, False

def extract_courses_info(html_chunk):
    # Parse the HTML chunk
    soup = BeautifulSoup(html_chunk, 'html.parser')

    # Find all <div> elements with class 'block' inside the HTML chunk
    course_divs = soup.find_all('div', class_='block')

    course_info_list = []  # Initialize an empty list to store course information

    for course_div in course_divs:
        # Extract course title and description
        title_elem = course_div.find('h1', class_='text-secondary')
        description_elem = course_div.find('div', class_='leading-relaxed')
        link_elem = course_div.find('a', href=True)

        if title_elem and description_elem:
            title = title_elem.text.strip()
            description = description_elem.text.strip()
            link = link_elem['href']

            # call the function and pass the link here
            some_more_data, successfactor = fetch_additional_course_details(link)

            if not successfactor:
                print(f"{title} has too little information.")
                continue

            # Create a dictionary to store the course details
            course_info = {
                'Title': title,
                'Description': description,
                'Link': link,
                'Details': some_more_data
            }

            course_info_list.append(course_info)

    return course_info_list

def scrape_courses(url):
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'html.parser')

        # Assuming the courses are inside one of the sections, 
        # we'll pass the entire page's HTML to extract_courses_info.
        list_of_courses = extract_courses_info(soup.prettify())

        return list_of_courses
    except Exception as e:
        print(f"An error occurred while scraping courses: {str(e)}")
        return []
# URL for page 1
url1 = 'https://moringaschool.com/courses/'
list_of_courses_page1 = scrape_courses(url1)

# URL for page 2
url2 = 'https://moringaschool.com/courses/page/2/'
list_of_courses_page2 = scrape_courses(url2)

# Combine the results if needed
all_courses = list_of_courses_page1 + list_of_courses_page2

# Print the courses
for i, course in enumerate(all_courses):
    print(i + 1, course)


The course has sections : 12 : https://moringaschool.com/courses/high-school-holiday-tech-bootcamp/
High School Holiday Tech Bootcamp has too little information.
The course has sections : 13 : https://moringaschool.com/courses/software-engineering-mobile/
The course has sections : 13 : https://moringaschool.com/courses/data-science-full-time-hybrid/
The course has sections : 13 : https://moringaschool.com/courses/cybersecurity-comptia-security-course/
The course has sections : 13 : https://moringaschool.com/courses/product-design-ux-ui-part-time-course/
The course has sections : 12 : https://moringaschool.com/courses/cybersecurity-prep/
Cybersecurity Prep has too little information.
The course has sections : 10 : https://moringaschool.com/courses/data-visualization-with-python/
Data Visualization with Python has too little information.
The course has sections : 13 : https://moringaschool.com/courses/devops-engineering/
The course has sections : 13 : https://moringaschool.com/courses/pr

In [7]:
DSFT = all_courses[1]
DSFT

{'Title': 'Data Science Full-time Hybrid',
 'Description': 'Are you passionate about data? Do you want to learn how to analyze data, and provide data-driven insights for decisions? This is the course for you. Kickstart the journey to your future career now!',
 'Link': 'https://moringaschool.com/courses/data-science-full-time-hybrid/',
 'Details': {'FAQs': 'Learn Python for Data Science, Data Analysis, Data Visualization, Data Modelling & Machine Learning in 25 weeks\n\n\nThe credits are rolling on a movie you’ve just concluded watching and Netflix automatically queues the preview for another movie that you might like; You’re doing some online shopping on Jumia and while scrolling down the product description, you notice a section that says “Most popular” that has the same product from different vendors at different prices; this is Data Science at work and businesses are always looking for talented individuals to help them analyze and draw actionable insights from their data in order to

In [8]:
DSPT = all_courses[7]
DSPT

{'Title': 'Data Science Part-time',
 'Description': 'This is for individuals with a demanding schedule. Learn how to explore, analyze, visualize, build models with data using programming languages, such as Python and R, and deploy models into applications',
 'Link': 'https://moringaschool.com/courses/data-science-course-part-time/',
 'Details': {'FAQs': 'Have a demanding schedule? Explore our part-time option\n\n\nLearn Python for Data Science, Data Analysis, Data Visualization, Data Modelling & Machine Learning in 35 weeks\nThe credits are rolling on a movie you’ve just concluded watching and Netflix automatically queues the preview for another movie that you might like; You’re doing some online shopping on Jumia and while scrolling down the product description, you notice a section that says “Most popular” that has the same product from different vendors at different prices; this is Data Science at work and businesses are always looking for talented individuals to help them analyze a

In [9]:
DSFTO = all_courses[8]
DSFTO

{'Title': 'Data Science Full-time Online',
 'Description': 'Are you passionate about data? Do you want to learn how to analyze data, and provide data-driven insights for decisions? This is the course for you. Kickstart the journey to your future career now!',
 'Link': 'https://moringaschool.com/courses/data-science-course/',
 'Details': {'FAQs': 'Learn Python for Data Science, Data Analysis, Data Visualization, Data Modelling & Machine Learning in 25 weeks\n\n\nThe credits are rolling on a movie you’ve just concluded watching and Netflix automatically queues the preview for another movie that you might like; You’re doing some online shopping on Jumia and while scrolling down the product description, you notice a section that says “Most popular” that has the same product from different vendors at different prices; this is Data Science at work and businesses are always looking for talented individuals to help them analyze and draw actionable insights from their data in order to make info

In [10]:
data_science = [all_courses[1], all_courses[7], all_courses[8]]

for file in data_science:
    print(file)


{'Title': 'Data Science Full-time Hybrid', 'Description': 'Are you passionate about data? Do you want to learn how to analyze data, and provide data-driven insights for decisions? This is the course for you. Kickstart the journey to your future career now!', 'Link': 'https://moringaschool.com/courses/data-science-full-time-hybrid/', 'Details': {'FAQs': 'Learn Python for Data Science, Data Analysis, Data Visualization, Data Modelling & Machine Learning in 25 weeks\n\n\nThe credits are rolling on a movie you’ve just concluded watching and Netflix automatically queues the preview for another movie that you might like; You’re doing some online shopping on Jumia and while scrolling down the product description, you notice a section that says “Most popular” that has the same product from different vendors at different prices; this is Data Science at work and businesses are always looking for talented individuals to help them analyze and draw actionable insights from their data in order to ma

In [11]:
data_science

[{'Title': 'Data Science Full-time Hybrid',
  'Description': 'Are you passionate about data? Do you want to learn how to analyze data, and provide data-driven insights for decisions? This is the course for you. Kickstart the journey to your future career now!',
  'Link': 'https://moringaschool.com/courses/data-science-full-time-hybrid/',
  'Details': {'FAQs': 'Learn Python for Data Science, Data Analysis, Data Visualization, Data Modelling & Machine Learning in 25 weeks\n\n\nThe credits are rolling on a movie you’ve just concluded watching and Netflix automatically queues the preview for another movie that you might like; You’re doing some online shopping on Jumia and while scrolling down the product description, you notice a section that says “Most popular” that has the same product from different vendors at different prices; this is Data Science at work and businesses are always looking for talented individuals to help them analyze and draw actionable insights from their data in orde

In [21]:
!pip install packaging==20.9



  Using cached packaging-20.9-py2.py3-none-any.whl (40 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 23.2
    Uninstalling packaging-23.2:
      Successfully uninstalled packaging-23.2


In [15]:
# Train Word2Vec model.

from gensim.models import Word2Vec

# Extract text from your data and concatenate Description and Details
documents = [course['Description'] + ' ' + course['Details']['FAQs'] + ' ' + course['Details']['course_details'] + ' ' + course['Details']['prerequisites'] for course in data_science]

# Tokenize the documents (split text into words)
tokenized_documents = [doc.split() for doc in documents]

# Build the vocabulary
model = Word2Vec(vector_size=200, sg=0, window=5, min_count=1)
model.build_vocab(tokenized_documents)

# Train the Word2Vec model
model.train(tokenized_documents, total_examples=len(tokenized_documents), epochs=20)


(21641, 31020)

In [16]:
model.wv.most_similar('Details')

[('the', 0.9962291121482849),
 ('Data', 0.9962099194526672),
 ('to', 0.996100902557373),
 ('a', 0.9959900975227356),
 ('business', 0.9958959817886353),
 ('in', 0.9958807826042175),
 ('for', 0.9957026243209839),
 ('Science', 0.995649516582489),
 ('and', 0.9956473112106323),
 ('data', 0.995642364025116)]

In [17]:
# Save the trained Word2Vec model.
model.wv.save_word2vec_format("Word2Vec_ds.txt")

In [18]:
# Compress the saved Word2Vec model
import gzip
fp = open("Word2Vec_ds.txt", 'rb')
data = fp.read()

bindata = bytearray(data)

with gzip.open("Word2Vec_ds.txt.gz", 'wb') as f:
    f.write(bindata)

In [8]:
# import spacy
# import numpy
# from gensim.models import KeyedVectors

# # Load your trained Word2Vec model
# word_vectors = KeyedVectors.load_word2vec_format('Word2Vec_ds.txt', binary=False)

# # Define the function to convert Word2Vec to spaCy
# def convert_word2vec_to_spacy(word2vec_model_path, output_path):
#     nlp = spacy.blank('en')
#     with open(word2vec_model_path, 'rb') as file_:
#         header = file_.readline()
#         nr_row, nr_dim = header.split()
#         nlp.vocab.reset_vectors(width=int(nr_dim))
#         for line_no, line in enumerate(file_):
#             line = line.rstrip().decode('utf8')
#             parts = line.rsplit(' ', int(nr_dim))
#             word = parts[0]
#             vector = numpy.asarray([float(v) for v in parts[1:]], dtype='f')
#             nlp.vocab.set_vector(word, vector)
#     nlp.to_disk(output_path)

# # Convert and save
# convert_word2vec_to_spacy('Word2Vec_ds.txt', 'D:\Moringa\projos\Capstone\self_model')


In [24]:
!pip install --upgrade sqlalchemy

